In [5]:
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
import keras.utils as ku 
import numpy as np

Using TensorFlow backend.


In [6]:
# data = open("andy.txt",encoding="utf8").read().replace('\n',' ')
filenames = ["andy.txt","greenmile.txt","It.txt"]
allN = ""
for file in filenames:
    data = open("NEU/data/"+file,encoding="utf8").read()
    data = data.replace("'","")
    allN += data

allN

'Rita Hayworth and the Shawshank Redemption\n\nTheres a guy like me in every state and federal prison in America, I guess--Im\nthe guy who can get it for you. Tailor made cigarettes, a bag of reefer if \nyoure\npartial to that, a bottle of brandy to celebrate your son or daughters high\n\nschool graduation, or anything else within reason, that is.  It wasnt always \nthat way.\n\nI came to Shawshank when I was just twenty, and I am one of the few people in\nour happy little family willing to own up to what he did.  I committed murder.  \nI\nput a large insurance policy on my wife -- who was three years older than I was \n--\nand then I fixed the brakes on the Chevrolet coupe her father had given us as a\nwedding present.  It worked out exactly as I had planned -- except I hadnt \nplanned\n\non her stopping to pick up the neighbor woman and the neighbor woman’s infant \nson\non their way down Castle Hill and into town.  The brakes let go and the car\n\ncrashed through the bushes at the e

In [7]:
tokenizer = Tokenizer()
def dataset_preparation(data):
    corpus = data.lower().split("\n")
    tokenizer.fit_on_texts(corpus)
    total_words = len(tokenizer.word_index) + 1
    input_sequences = []
    for line in corpus:
        token_list = tokenizer.texts_to_sequences([line])[0]
        for i in range(1, len(token_list)):
            n_gram_sequence = token_list[:i+1]
            input_sequences.append(n_gram_sequence)
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))
    predictors, label = input_sequences[:,:-1], input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors,label,max_sequence_len,total_words

In [8]:
def create_model(predictors, label, max_sequence_len, total_words):
    input_len = max_sequence_len - 1
    model = Sequential()
    model.add(Embedding(total_words, 10, input_length=input_len))
    model.add(LSTM(150))
    model.add(Dropout(0.1))
    model.add(Dense(total_words, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam')
    model.fit(predictors, label, epochs=100, verbose=1)
    return model

In [9]:
def generate_text(seed_text, next_words, max_sequence_len, model):
    for j in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen= 
                             max_sequence_len-1, padding='pre')
        predicted = model.predict_classes(token_list, verbose=0)
  
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

In [11]:
X, Y, max_len, total_words = dataset_preparation(data)
model = create_model(X, Y, max_len, total_words)

C:\Users\Karti\Anaconda3\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:424: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Epoch 1/100
21390/21390 [==============================] - 27s 1ms/step - loss: 6.7526
Epoch 2/100
21390/21390 [==============================] - 26s 1ms/step - loss: 6.4851
Epoch 3/100
21390/21390 [==============================] - 26s 1ms/step - loss: 6.2770
Epoch 4/100
21390/21390 [==============================] - 26s 1ms/step - loss: 6.1478
Epoch 5/100
21390/21390 [==============================] - 26s 1ms/step - loss: 6.0213
Epoch 6/100
21390/21390 [==============================] - 27s 1ms/step - loss: 5.8980
Epoch 7/100
21390/21390 [==============================] - 27s 1ms/step - loss: 5.7574
Epoch 8/100
21390/21390 [==============================] - 27s 1ms/step - loss: 5.5908
Epoch 9/100
21390/21390 [==============================] - 27s 1ms/step - loss: 5.4032
Epoch 10/100
21390/21390 [==============================] - 27s 1ms/step - loss: 5.2088
Epoch 11/100
21390/21390 [==============================] - 26s 1ms/step - loss: 5.0137
Epoch 12/100
21390/21390 [===============

21390/21390 [==============================] - 29s 1ms/step - loss: 0.7652
Epoch 94/100
21390/21390 [==============================] - 29s 1ms/step - loss: 0.7560
Epoch 95/100
21390/21390 [==============================] - 28s 1ms/step - loss: 0.7475
Epoch 96/100
21390/21390 [==============================] - 28s 1ms/step - loss: 0.7412
Epoch 97/100
21390/21390 [==============================] - 27s 1ms/step - loss: 0.7266
Epoch 98/100
21390/21390 [==============================] - 28s 1ms/step - loss: 0.7122
Epoch 99/100
21390/21390 [==============================] - 28s 1ms/step - loss: 0.7118
Epoch 100/100
21390/21390 [==============================] - 29s 1ms/step - loss: 0.6999


In [19]:
text = generate_text("Andy was convicted to lifetime imprisonment because", 15, max_len, model)
print(text)

Andy was convicted to lifetime imprisonment because the stupendous changes had began to better off by the love or a man who


In [32]:
text = generate_text("the prison made him feel like", 15, max_len, model)
print(text)

the prison made him feel like it it was good for the third time i know it he felt at a


In [33]:
text = generate_text("Hadley was", 16, max_len, model)
print(text)

Hadley was a foolhardy move not that stanley should not have his own business—god forbid he should not


In [42]:
text = generate_text("Andy and I are in prison", 16, max_len, model)
print(text)

Andy and I are in prison ” he said he was bored at this with her in his arms and the business


In [46]:
text = generate_text("In prison Andy would only", 16, max_len, model)
print(text)

In prison Andy would only be known of what as there were still enough over the tub holding the left time


In [49]:
text = generate_text("If i tell you about Shawshank redemption", 15, max_len, model)
print(text)

If i tell you about Shawshank redemption really really i know i’ll hurt she had been a bad ” a hat that


In [50]:
text = generate_text("Brooks was here", 16, max_len, model)
print(text)

Brooks was here when he said ” chief said interjected it to all in the car that which had


In [51]:
text = generate_text("Ellis is", 16, max_len, model)
print(text)

Ellis is me there were blue tattoos on the arms—arcane symbols which he had still the canal in


In [52]:
text = generate_text("How was Andy as a person?", 16, max_len, model)
print(text)

How was Andy as a person? he told the police he had been a joke between them—the them—the a great with mystik


In [58]:
text = generate_text("Andy was troubled by", 16, max_len, model)
print(text)

Andy was troubled by we said he was going to be able to make it some of the head and


In [63]:
text = generate_text("Andy said whatever", 18, max_len, model)
print(text)

Andy said whatever not you have like you ” don asked the voice george and got the three time that that


In [64]:
text = generate_text("Rita Hayworth and Shawshank Redemption", 100, max_len, model)
print(text)

Rita Hayworth and Shawshank Redemption really said i have don’t know ” garton said he was bored up again again not not not a pale statue with a sandwich ” hagarty said who think it was going to be all a strip bowl with it no 000 a job cysts a b beating up the rim of the past hat george went up the side of the side of his shirt and his and all all struck him as if that that that had father see the way and it was doing that he would been mellow at the front of the pants who she


In [53]:
text = generate_text("Rita Hayworth and the Shawshank Redemption", 10000, max_len, model)
print(text)

Rita Hayworth and the Shawshank Redemption right out of the infirmary and the master who had been out of the far club of the month nice like my mind is as the rock red outer other three years in all at his wife 30 found a birth before that night he can young way andy turns over my head into the river no i weak the night there or a big stock man at a small pipe like country by the kid on the next if he put him we his friend had a friend man in the infirmary to some five worth minutes i only now in the wall the rest of camels to keep your new a little man with the window of the end of his mind if he done a few just from my kid clear that the prison woman by his time before i did is at that much as ive been taken to him if dont remember to be him strong woman as andy thought it was wed be for gratitude in the nine or dutch wife like posters ive could be to sixteen to the so track i might have how the long or six or the fisherman time to me about a man at the time if it is i guess for how 

In [66]:
text = generate_text("Andy helped", 100, max_len, model)
print(text)

Andy helped the bus station and kept that that he told the third of the two of glointon and grew fat jeans then a week of the left and at george at that that had still been a mermaid and with an bunch to a job pack of muh muh matches ” ” it he was still the smell with a sandwich t bowl that forty got over the last of the parlor struck the parlor that thousands of them that maybe that that then then then he had been a bad in a surprise she for the week of a town


In [68]:
text = generate_text("I liked Andy because", 20, max_len, model)
print(text)

I liked Andy because for the clown again and although avarino could sink the tail of his shirt then then it would been one


In [69]:
text = generate_text("Greenmile", 10000, max_len, model)
print(text)

Greenmile summer crowded beach must sound to an exhausted swimmer who struggles with the undertow then had made it had still no liking for to her home she could seen it else he would been swept most of the others most most of a pitcher and and but at the first of front thousands of the company—starting then she seemed to his eyes listening had 000 a hundred crazies from a week and them—the up a instant of the parlor and screamed at the company—starting salary 30 000 to him barney that a job big a strip beating crazies a hipshot way of a college and i put it right maybe that’s wrong why a heart bill really will no bad we where no i had seen no reason a bright shade he was it and fine tried he was still that who sometimes had have been able to get all a dozen thing as a b move a hole that that that’s the break that got it to still such that he would have she had need an car and really floating a surface of the parlor however you think going to think his heart was on her mother she could 